In [1]:
import torch
import onnx
import onnxruntime as ort
import netron

In [2]:
print(torch.__version__)
print(onnx.__version__)
print(ort.get_device())
print(ort.__version__)

2.7.0+cu126
1.18.0
GPU
1.22.0


In [3]:
MODEL_PATH = "./result/efficientnet_v2_l_best.pt"
ONNX_PATH = "./onnx/efficientnet_v2_l.onnx"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

cuda


In [4]:
model = torch.load(MODEL_PATH, map_location=DEVICE, weights_only=False)
model.to(DEVICE)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

In [5]:
# 더미 생성: (batch, 1, mel_bins, time)
dummy = torch.randn(1, 1, 64, 101, device=DEVICE)

In [6]:
# onnx 모델 변환
print("ONNX 변환중...")

torch.onnx.export(model, # PyTorch 모델
                dummy, # 입력 텐서 (튜플도 가능)
                ONNX_PATH, # 저장할 파일 경로
                export_params=True, # 모델 파라미터 포함
                opset_version=11, # onnx의 연산자 버전
                input_names=['input'], # 입력 이름
                output_names=['output'], # 출력 이름
                dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}} # 배치 크기를 동적으로 지정
)

print(f"ONNX 변환완료'{ONNX_PATH}'에 저장됨")

ONNX 변환중...
ONNX 변환완료'./onnx/efficientnet_v2_l.onnx'에 저장됨


In [7]:
model = onnx.load(ONNX_PATH)
print(onnx.helper.printable_graph(model.graph))

graph main_graph (
  %input[FLOAT, batch_sizex1x64x101]
) initializers (
  %features.4.0.block.2.fc1.weight[FLOAT, 24x384x1x1]
  %features.4.0.block.2.fc1.bias[FLOAT, 24]
  %features.4.0.block.2.fc2.weight[FLOAT, 384x24x1x1]
  %features.4.0.block.2.fc2.bias[FLOAT, 384]
  %features.4.1.block.2.fc1.weight[FLOAT, 48x768x1x1]
  %features.4.1.block.2.fc1.bias[FLOAT, 48]
  %features.4.1.block.2.fc2.weight[FLOAT, 768x48x1x1]
  %features.4.1.block.2.fc2.bias[FLOAT, 768]
  %features.4.2.block.2.fc1.weight[FLOAT, 48x768x1x1]
  %features.4.2.block.2.fc1.bias[FLOAT, 48]
  %features.4.2.block.2.fc2.weight[FLOAT, 768x48x1x1]
  %features.4.2.block.2.fc2.bias[FLOAT, 768]
  %features.4.3.block.2.fc1.weight[FLOAT, 48x768x1x1]
  %features.4.3.block.2.fc1.bias[FLOAT, 48]
  %features.4.3.block.2.fc2.weight[FLOAT, 768x48x1x1]
  %features.4.3.block.2.fc2.bias[FLOAT, 768]
  %features.4.4.block.2.fc1.weight[FLOAT, 48x768x1x1]
  %features.4.4.block.2.fc1.bias[FLOAT, 48]
  %features.4.4.block.2.fc2.weight[FLOAT,

In [8]:
try:
    onnx.checker.check_model(model)
    print("모델 검증 성공")
except onnx.checker.ValidationError as e:
    print("모델 검증 실패")
    print(e)

모델 검증 성공


In [9]:
# 모델 시각화
netron.start (ONNX_PATH)

Serving './onnx/efficientnet_v2_l.onnx' at http://localhost:8080


('localhost', 8080)